# Compare spread and eigenvalue centrality when choosing vertices to remove

On several test graphs, use both centrality measures to select a fixed number vertices to remove (where the number is selected based on the size of the graph). 

In [1]:
import igraph
from scipy.io import mmread
from scipy.linalg import eigh
import numpy as np
from dissim.igraph_util import nodes_from_igraph, colors_from_nodes
from dissim.main import iterate, DSState
from dissim.death_ftn import generate_normal_death_ftn_factory
from dissim.rng import SimpleSampler
from dissim.centrality import compute_spread_centrality_of_matrix, compute_eigenvector_centrality_of_matrix
import matplotlib.pyplot as plt

In [2]:
def largest_eigenvalue(adj: np.array) -> float:
    n = adj.shape[0]
    w = eigh(adj, eigvals_only=True, subset_by_index=[n - 1, n - 1])
    return np.real(w)[0]

In [3]:
def remove_vertex(adj, i):
    result = np.delete(adj, i, axis=0)
    result = np.delete(result, i, axis=1)
    return result

## 1. Karate graph

In [17]:
matrix = mmread("/home/rkingan/winhome/Documents/dev/src/sandra-math5001-2022/soc-karate.mtx")
adj = matrix.todense()
n = adj.shape[0]
to_remove = 17 # half of the graph
print(n)
print(np.sum(adj))

34
156.0


### Spread centrality

In [5]:
sampler = SimpleSampler(42)

cent = compute_spread_centrality_of_matrix(adj)
cent_pairs = sampler([(i, cent[i]) for i in range(n)], n)
cent_pairs.sort(key=lambda t: -t[1])
removals = cent_pairs[:to_remove]
ss_removal_indices = sorted([t[0] for t in removals])
minor = np.delete(adj, ss_removal_indices, axis=0)
minor = np.delete(minor, ss_removal_indices, axis=1)
ss_largest_eigenvalue_after = largest_eigenvalue(minor)

print(",".join(str(x) for x in ss_removal_indices))
print(ss_largest_eigenvalue_after)

0,1,2,3,7,8,9,13,19,23,27,28,29,30,31,32,33
2.4811943040920155


### Eigenvector centrality

In [6]:
sampler = SimpleSampler(42)

cent = compute_eigenvector_centrality_of_matrix(adj)
cent_pairs = sampler([(i, cent[i]) for i in range(n)], n)
cent_pairs.sort(key=lambda t: -t[1])
removals = cent_pairs[:to_remove]
eig_removal_indices = sorted([t[0] for t in removals])
minor = np.delete(adj, eig_removal_indices, axis=0)
minor = np.delete(minor, eig_removal_indices, axis=1)
eig_largest_eigenvalue_after = largest_eigenvalue(minor)

print(",".join(str(x) for x in eig_removal_indices))
print(eig_largest_eigenvalue_after)

0,1,2,3,7,8,9,13,19,23,27,28,29,30,31,32,33
2.4811943040920155


In [12]:
set(ss_removal_indices) - set(eig_removal_indices), set(eig_removal_indices) - set(ss_removal_indices)

({69, 92}, {45, 89})

## 2. Football graph

In [7]:
G = igraph.Graph.Read_GML("/home/rkingan/winhome/Documents/dev/src/sandra-math5001-2022/Graphs-Large-02/football.gml").as_undirected()

In [18]:
adj = np.array(G.get_adjacency().data)
n = adj.shape[0]
to_remove = int(n / 2)
print(n, len(G.es), to_remove)

115 613 57


### Spread centrality

In [9]:
sampler = SimpleSampler(42)

cent = compute_spread_centrality_of_matrix(adj)
cent_pairs = sampler([(i, cent[i]) for i in range(n)], n)
cent_pairs.sort(key=lambda t: -t[1])
removals = cent_pairs[:to_remove]
ss_removal_indices = sorted([t[0] for t in removals])
minor = np.delete(adj, ss_removal_indices, axis=0)
minor = np.delete(minor, ss_removal_indices, axis=1)
ss_largest_eigenvalue_after = largest_eigenvalue(minor)

print(",".join(str(x) for x in ss_removal_indices))
print(ss_largest_eigenvalue_after)

0,1,2,3,4,5,6,7,8,9,10,13,15,16,21,22,23,25,32,39,40,41,46,47,48,49,51,52,53,60,64,67,68,69,72,73,74,77,78,80,81,82,83,84,88,92,93,98,100,102,104,106,107,108,110,111,114
9.056820324376668


### Eigenvector centrality

In [10]:
sampler = SimpleSampler(42)

cent = compute_eigenvector_centrality_of_matrix(adj)
cent_pairs = sampler([(i, cent[i]) for i in range(n)], n)
cent_pairs.sort(key=lambda t: -t[1])
removals = cent_pairs[:to_remove]
eig_removal_indices = sorted([t[0] for t in removals])
minor = np.delete(adj, eig_removal_indices, axis=0)
minor = np.delete(minor, eig_removal_indices, axis=1)
eig_largest_eigenvalue_after = largest_eigenvalue(minor)

print(",".join(str(x) for x in eig_removal_indices))
print(eig_largest_eigenvalue_after)

0,1,2,3,4,5,6,7,8,9,10,13,15,16,21,22,23,25,32,39,40,41,45,46,47,48,49,51,52,53,60,64,67,68,72,73,74,77,78,80,81,82,83,84,88,89,93,98,100,102,104,106,107,108,110,111,114
9.091798552807656
